In [1]:
!pip install git+https://github.com/CahanLab/oneSC.git

  Cloning https://github.com/CahanLab/oneSC.git to /tmp/pip-req-build-s_1p35kb
  Running command git clone --filter=blob:none --quiet https://github.com/CahanLab/oneSC.git /tmp/pip-req-build-s_1p35kb
  Resolved https://github.com/CahanLab/oneSC.git to commit dacd08d8f750d10e3c4685b394ed046d9fd4ba7c
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.0/522.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.3 MB/s et

In [ ]:
!pip install cairocffi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
!pip install pycairo

  Using cached pycairo-1.26.1.tar.gz (346 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pycairo (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pycairo
Failed to build pycairo
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (pycairo)


In [3]:
!pip install tqdm_joblib

This is Method 1 of inferring networks

In [1]:
import numpy as np
import pandas as pd
import onesc
import networkx as nx
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import os

/usr/local/lib/python3.10/dist-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
import time

In [3]:
train_exp = pd.read_csv("https://cnobjects.s3.amazonaws.com/OneSC/Pual_2015/train_exp.csv", index_col = 0)
samp_tab = pd.read_csv("https://cnobjects.s3.amazonaws.com/OneSC/Pual_2015/samp_tab.csv", index_col = 0)
pt_col = 'dpt_pseudotime'
cluster_col = 'cell_types'

In [4]:
initial_clusters = ['CMP']
end_clusters = ['Erythrocytes', 'Granulocytes', 'Monocytes', 'MK']

In [5]:
state_path = onesc.construct_cluster_network(train_exp, samp_tab, initial_clusters = initial_clusters, terminal_clusters = end_clusters, cluster_col = cluster_col, pseudo_col = pt_col)


In [6]:
lineage_cluster = onesc.extract_trajectory(state_path,initial_clusters, end_clusters)


In [7]:
# find the boolean threshold for each gene
vector_thresh = onesc.find_threshold_vector(train_exp, samp_tab, cluster_col = "cell_types", cutoff_percentage=0.4)

# identify the finner time steps at which genes change along individual trajectory
lineage_time_change_dict = onesc.find_gene_change_trajectory(train_exp, samp_tab, lineage_cluster, cluster_col, pt_col, vector_thresh, pseudoTime_bin=0.01)

In [8]:
# define boolean states profiles for each cell cluster
state_dict = onesc.define_states(train_exp, samp_tab, lineage_cluster, vector_thresh, cluster_col, percent_exp = 0.3)

# save the dictionary of Boolean states into a pickle object.
# we will be needing the Boolean profiles of initial state for running simulations
pickle.dump(state_dict, open("state_dict.pickle", "wb"))

# define transition profiles for each cell clusters
transition_dict = onesc.define_transition(state_dict)

In [9]:
# curate the training data for GRN inference for each gene
training_data = onesc.curate_training_data(state_dict, transition_dict, lineage_time_change_dict, samp_tab, cluster_id = cluster_col, pt_id = pt_col,act_tolerance = 0.04)

# calculate the pearson correlation between genes. This adds more information during the inference step.
corr_mat = onesc.calc_corr(train_exp)
ideal_edge_num = round(0.4 * corr_mat.shape[1])


Cebpe have conflicting states, the below states are deleted
['GMP_unstable_trajectory_1', 'GMP_stable_trajectory_2']
Fli1 have conflicting states, the below states are deleted
['CMP_stable_trajectory_0', 'CMP_stable_trajectory_1', 'CMP_stable_trajectory_2', 'CMP_unstable_trajectory_3']
Gata1 have conflicting states, the below states are deleted
['CMP_unstable_trajectory_0', 'CMP_stable_trajectory_1', 'CMP_stable_trajectory_2', 'CMP_stable_trajectory_3']
Gata2 have conflicting states, the below states are deleted
['CMP_stable_trajectory_0', 'CMP_unstable_trajectory_1', 'CMP_unstable_trajectory_2', 'CMP_stable_trajectory_3']
Sox4 have conflicting states, the below states are deleted
['CMP_stable_trajectory_1', 'CMP_stable_trajectory_2', 'CMP_unstable_trajectory_3']


In [10]:
start_time = time.time()
grn_ensemble = onesc.create_network_ensemble(training_data,
                                            corr_mat,
                                            ideal_edges = ideal_edge_num,
                                            num_generations = 300,
                                            max_iter = 30,
                                            num_parents_mating = 4,
                                            run_parallel = False,
                                            sol_per_pop = 30,
                                            reduce_auto_reg = True,
                                            GA_seed_list = [1, 2, 3, 4, 5],
                                            init_pop_seed_list = [21, 22, 23, 24, 25]) # this would generate 25 networks (one for each GA, init seed combo)

time_lapse = time.time() - start_time

/usr/local/lib/python3.10/dist-packages/onesc/genetic_algorithm_GRN_trimming.py:845: UserWarning: Maximum number of cores is 2
  warnings.warn("Maximum number of cores is " + str(cpu_count()))
Network inference ensemble (no parallelization): 100%|██████████| 25/25 [3:33:19<00:00, 511.98s/it]


In [11]:
print("OneSC training takes " + str(time_lapse))

OneSC training takes 12800.893421649933
